In [ ]:
# langcahianライブラリのインストール
!pip install langchain langchain-community

# openaiライブラリのインストール
!pip install openai

# lnagchainでopenaiを使うためのライブラリをインストール
!pip install langchain_openai

 #pdfを読み込むためのライブラリをインストールする
!pip install pypdf

# embeddingで利用するライブラリのインストール
!pip install tiktoken

# ローカルマシン上で実行できるchromaベクターデータベースのインストール
!pip install chromadb

# 簡単なGUIを作成できるライブラリのインストール
!pip install gradio

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

#APIキーの登録
os.environ["OPENAI_API_KEY"] = "__YOUR KEY__"

url = "https://api.openai.iniad.org/api/v1/"

# OpenAI埋め込みモデルのインスタンスを作成
embeddings_model = OpenAIEmbeddings(
    openai_api_base= url
)


# PDFファイルの読み込み
loader = PyPDFLoader("Disease.pdf")
text = "\n".join([page.page_content for page in loader.load()])

# Text Splitterの設定
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    add_start_index=True
)

# テキストの分割
documents = text_splitter.create_documents([text])

# ベクトル化したテキストをChromaDBに保存する
db = Chroma.from_documents(documents,embeddings_model)

In [ ]:
query = "インフルエンザの症状を教えてください"
docs = db.similarity_search(query)
print(docs)

In [ ]:
embedding_vector = OpenAIEmbeddings(openai_api_base= url).embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs)

In [ ]:
# 必要なライブラリをインポートする
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# retriever(検索対象のVectorDB)の定義
retriever = db.as_retriever()

# テンプレートを定義
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
# promptを定義。これがLLMの入力になる
prompt = ChatPromptTemplate.from_template(template)

# LLMを定義。今回はChatGPTの4o-miniを利用する
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    openai_api_base= url,
    verbose=True
    )


def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Chainを定義。これはおまじないだと思って下さい。
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser(verbose=True)
    )

# 問い合わせしたい内容を定義
input = "インフルエンザの症状を教えてください"

references = retriever.get_relevant_documents(input)
output_by_retriever = chain.invoke(input)
output_by_simple_llm = llm.invoke(input)

# 参考にした箇所を表示します。
# 情報の取得元や参照した文献など、生成された回答の背景にある情報源が含まれています
print("参考にした箇所: " + str(references))


# 検索結果を基に生成された回答や内容が含まれています
print("RAGの出力結果: " + str(output_by_retriever))

# シンプルな言語モデルのみを使用して生成された回答や内容を含んでいます
print("シンプルなLLM（ChatGPT）の出力結果: " + str(output_by_simple_llm))

In [ ]:
import gradio as gr
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# ドキュメントの読み込みとベクトルストアの作成
retriever = db.as_retriever(search_kwargs={"k": 3})

# テンプレートとプロンプトの定義
template_with_context = """以下のcontextのみに基づいて質問にできるだけ詳しく箇条書きで答えなさい。:
{context}
質問: {question}
"""
template_without_context = """質問にできるだけ詳しく箇条書きで答えなさい。:
質問: {question}
"""

prompt_with_context = ChatPromptTemplate.from_template(template_with_context)
prompt_without_context = ChatPromptTemplate.from_template(template_without_context)

# LLMの定義（ストリーミング対応）
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    openai_api_base= url,
    verbose=True)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# Chainの定義
chain_with_context = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_with_context
    | llm
    | StrOutputParser()
)

chain_without_context = (
    {"question": RunnablePassthrough()}
    | prompt_without_context
    | llm
    | StrOutputParser()
)

def process_query_with_rag(query):
    # 関連ドキュメントの取得
    relevant_docs = retriever.get_relevant_documents(query)
    sources = [doc.page_content for doc in relevant_docs]

    # 回答の生成（ストリーミング）
    answer = ""
    for chunk in chain_with_context.stream(query):
        answer += chunk
        yield answer, "\n\n".join(sources)

def process_query_without_rag(query):
    # 回答の生成（ストリーミング）
    answer = ""
    for chunk in chain_without_context.stream(query):
        answer += chunk
        yield answer, "なし"

# Gradioインターフェースの定義
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("## RAGあり")
            input_text_with_rag = gr.Textbox(lines=2, placeholder="質問を入力してください...")
            output_text_with_rag = gr.Markdown(label="回答")
            sources_with_rag = gr.Textbox(label="参照ソース")
            rag_button = gr.Button("RAGありで質問する")
            rag_button.click(fn=process_query_with_rag, inputs=input_text_with_rag, outputs=[output_text_with_rag, sources_with_rag])

        with gr.Column():
            gr.Markdown("## RAGなし")
            input_text_without_rag = gr.Textbox(lines=2, placeholder="質問を入力してください...")
            output_text_without_rag = gr.Markdown(label="回答")
            sources_without_rag = gr.Textbox(label="参照ソース")
            no_rag_button = gr.Button("RAGなしで質問する")
            no_rag_button.click(fn=process_query_without_rag, inputs=input_text_without_rag, outputs=[output_text_without_rag, sources_without_rag])

demo.launch()